<a href="https://colab.research.google.com/github/anuraagkansara/SecureNLP/blob/master/code/CD_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pytorch-pretrained-bert

In [0]:
import pandas as pd
import sklearn
import torch
import pickle

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
from tools import *
import convert_examples_to_features

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
train_df = pd.read_csv('/content/train.csv', header=None)
train_df.head()

,0,1
0,1,APT3 ( also known UPS ) actors responsible Ope...
1,0,This actor initiated recent campaign November ...
2,0,The attacker leveraged multiple exploits targe...
3,1,CVE-2014-6332 disclosed publicly 2014-11-11 Wi...
4,1,CVE-2014- 4113 privilege escalation vulnerabil...


In [8]:
train_df[train_df.isna().any(axis=1)]

,0,1


In [9]:
dev_df = pd.read_csv('/content/test2.csv', header=None)
dev_df.head()


,0,1
0,0,Low - profile information - stealing Trojan us...
1,0,Symantec uncovered elusive Trojan used cyberes...
2,0,Seaduke ( detected Symantec Trojan Seaduke ) l...
3,0,Seaduke used attacks number major government -...
4,0,The malware hides behind numerous layers encry...


In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [11]:
train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df[0],
    'alpha':['a']*train_df.shape[0],
    'text': train_df[1].replace(r'\n', ' ', regex=True)
})

train_df_bert.head()

,id,label,alpha,text
0,0,1,a,APT3 ( also known UPS ) actors responsible Ope...
1,1,0,a,This actor initiated recent campaign November ...
2,2,0,a,The attacker leveraged multiple exploits targe...
3,3,1,a,CVE-2014-6332 disclosed publicly 2014-11-11 Wi...
4,4,1,a,CVE-2014- 4113 privilege escalation vulnerabil...


In [12]:
dev_df_bert = pd.DataFrame({
    'id':range(len(dev_df)),
    'label':dev_df[0],
    'alpha':['a']*dev_df.shape[0],
    'text': dev_df[1].replace(r'\n', ' ', regex=True)
})

dev_df_bert.head()

,id,label,alpha,text
0,0,0,a,Low - profile information - stealing Trojan us...
1,1,0,a,Symantec uncovered elusive Trojan used cyberes...
2,2,0,a,Seaduke ( detected Symantec Trojan Seaduke ) l...
3,3,0,a,Seaduke used attacks number major government -...
4,4,0,a,The malware hides behind numerous layers encry...


In [13]:
len(train_df_bert)

12904

In [0]:
train_df_bert.to_csv('/content/train.tsv', sep='\t', index=False, header=False)

In [0]:
dev_df_bert.to_csv('/content/dev.tsv', sep='\t', index=False, header=False)

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [0]:
# !ls

In [0]:
# !unzip /content/drive/"My Drive"/upload_bert.zip

In [0]:
# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = "/content/"

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'SecureNLP'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'/content/outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'/content/reports/{TASK_NAME}_evaluation_report/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = '/content/cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 5
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [0]:
output_mode = OUTPUT_MODE # CLASSIFICATION MODE

cache_dir = CACHE_DIR

In [0]:
#REPORT DIRECTORY STRUCTURE CREATION, IF NOT CREATED
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)

In [0]:
#OUTPUT DIRECTORY STRUCTURE CREATION, IF NOT CREATED
if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(OUTPUT_DIR))
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [0]:
#BinaryClassificationProcessor in tools.py
processor = BinaryClassificationProcessor()
# aim is to convert each tuple to an InputExample Object guid,text_a,text_b, label
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

In [24]:
train_examples_len

12904

In [0]:
label_list = processor.get_labels() # [0, 1] for binary classification
# length =2
num_labels = len(label_list)

In [26]:
num_labels

2

In [0]:
# (12904/24/1)*5
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [28]:
num_train_optimization_steps

2685

In [29]:
# Load pre-trained model tokenizer (vocabulary)
# Runs end-to-end tokenization: punctuation splitting + wordpiece
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmphlvhl3fu
100%|██████████| 213450/213450 [00:00<00:00, 5358131.92B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmphlvhl3fu to cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmphlvhl3fu
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c

In [0]:
label_map = {label: i for i, label in enumerate(label_list)}
#for each tuple, appending static data (awkward) 
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

In [0]:
#label_map
#train_examples_for_processing
#cpu_count()
#convert_example_to_feature(train_examples_for_processing[0])


In [31]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
       #imap() or any mapping function takes one value and 'maps' it to another via the function's return value.
       #tqdm:: Experimental IPython/Jupyter Notebook widget 
        train_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

Preparing to convert 12904 examples..
Spawning 1 processes..


In [32]:
#train_features contains list of InputFeatures object (input_ids,input_mask,segment_ids,label_id)
train_features[1].label_id

0

In [0]:
# Write the pickled representation of the object obj to the open file object file
with open(DATA_DIR + "train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

In [34]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)
# model = BertForSequenceClassification.from_pretrained(CACHE_DIR + 'cased_base_bert_pytorch.tar.gz', cache_dir=CACHE_DIR, num_labels=num_labels)

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz not found in cache, downloading to /tmp/tmpwz75ryma
100%|██████████| 404400730/404400730 [00:04<00:00, 81059674.17B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpwz75ryma to cache at /content/cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /content/cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpwz75ryma
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at /content/cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a

In [35]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']


In [37]:
print(param_optimizer[0])

('bert.embeddings.word_embeddings.weight', Parameter containing:
tensor([[-0.0005, -0.0416,  0.0131,  ..., -0.0039, -0.0335,  0.0150],
        [ 0.0169, -0.0311,  0.0042,  ..., -0.0147, -0.0356, -0.0036],
        [-0.0006, -0.0267,  0.0080,  ..., -0.0100, -0.0331, -0.0165],
        ...,
        [-0.0064,  0.0166, -0.0204,  ..., -0.0418, -0.0492,  0.0042],
        [-0.0048, -0.0027, -0.0290,  ..., -0.0512,  0.0045, -0.0118],
        [ 0.0313, -0.0297, -0.0230,  ..., -0.0145, -0.0525,  0.0284]],
       device='cuda:0', requires_grad=True))


In [0]:
#filtering out parameters with no_decay
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [0]:
#optimizer_grouped_parameters[0]

In [0]:
# The differences with PyTorch Adam optimizer are the following:

# BertAdam implements weight decay fix,
# BertAdam doesn't compensate for bias as in the regular Adam optimizer.
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=LEARNING_RATE,
                     warmup=WARMUP_PROPORTION,
                     t_total=num_train_optimization_steps)

In [0]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [42]:
logger.info("***** Running training *****")
logger.info("  Num examples = %d", train_examples_len)
logger.info("  Batch size = %d", TRAIN_BATCH_SIZE)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.float)

INFO:root:***** Running training *****
INFO:root:  Num examples = 12904
INFO:root:  Batch size = 24
INFO:root:  Num steps = 2685


In [0]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data) #same no. of train_data
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [44]:
len(train_dataloader)

538

In [45]:
model.train()
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        logits = model(input_ids, segment_ids, input_mask, labels=None)

        if OUTPUT_MODE == "classification":
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
        elif OUTPUT_MODE == "regression":
            loss_fct = MSELoss()
            loss = loss_fct(logits.view(-1), label_ids.view(-1))

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

0.271391

Epoch:  20%|██        | 1/5 [05:44<22:59, 344.82s/it]

0.217854


0.113782

...

0.003691

0.178353

0.122903

Epoch: 100%|██████████| 5/5 [29:33<00:00, 354.08s/it]

0.001300


In [46]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(OUTPUT_DIR)

'/content/outputs/SecureNLP/vocab.txt'

In [47]:
%cd /content/outputs/SecureNLP/ 
!pwd

/content/outputs/SecureNLP
/content/outputs/SecureNLP


In [0]:
import tarfile

tf = tarfile.open("SecureNLP.tar.gz", mode = "w:gz")
tf.add("config.json")
tf.add("pytorch_model.bin")
tf.close()

In [49]:
%cd ../../
!pwd

/content
/content


In [0]:
# tf=tarfile.open("SecureNLP.tar.gz")
# tf.extractall()

In [0]:
# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'SecureNLP.tar.gz'


In [0]:
def get_eval_report(task_name, labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "task": task_name,
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }

def compute_metrics(task_name, labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(task_name, labels, preds)

In [54]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR + 'vocab.txt', do_lower_case=False)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file outputs/SecureNLP/vocab.txt


In [55]:
type(tokenizer)

pytorch_pretrained_bert.tokenization.BertTokenizer

In [0]:
processor = BinaryClassificationProcessor()
eval_examples = processor.get_dev_examples(DATA_DIR)
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
eval_examples_len = len(eval_examples)

In [0]:
label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in eval_examples]

In [58]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

Preparing to convert 618 examples..
Spawning 1 processes..


In [0]:
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)

In [0]:
if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.float)

In [0]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

In [62]:
!ls 
!pwd

cache				 outputs      sample_data  train.csv
convert_examples_to_features.py  __pycache__  test2.csv    train_features.pkl
dev.tsv				 reports      tools.py	   train.tsv
/content


In [63]:
#ref='/content/drive/My Drive/Study/SecureNLP.tar.gz'
ref='/content/outputs/SecureNLP/SecureNLP.tar.gz'

model = BertForSequenceClassification.from_pretrained(ref, cache_dir=CACHE_DIR, num_labels=len(label_list))

INFO:pytorch_pretrained_bert.modeling:loading archive file /content/outputs/SecureNLP/SecureNLP.tar.gz
INFO:pytorch_pretrained_bert.modeling:extracting archive file /content/outputs/SecureNLP/SecureNLP.tar.gz to temp dir /tmp/tmpld3wjbj5
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



In [64]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
from sklearn.metrics import *

In [66]:
model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []
import numpy as np

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    # create eval loss and other metric required by the task
    if OUTPUT_MODE == "classification":
        loss_fct = CrossEntropyLoss()
        tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
    elif OUTPUT_MODE == "regression":
        loss_fct = MSELoss()
        tmp_eval_loss = loss_fct(logits.view(-1), label_ids.view(-1))

    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps
preds = preds[0]
if OUTPUT_MODE == "classification":
    preds = np.argmax(preds, axis=1)
elif OUTPUT_MODE == "regression":
    preds = np.squeeze(preds)
result = compute_metrics(TASK_NAME, all_label_ids.numpy(), preds)

result['eval_loss'] = eval_loss

output_eval_file = os.path.join(REPORTS_DIR, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in (result.keys()):
        logger.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:root:***** Eval results *****
INFO:root:  task = SecureNLP
INFO:root:  mcc = 0.9620048366803099
INFO:root:  tp = 89
INFO:root:  tn = 523
INFO:root:  fp = 5
INFO:root:  fn = 1
INFO:root:  eval_loss = 0.03151033365000517


In [0]:
tp=result['tp']
fp=result['fp']
tn=result['tn']
fn=result['fn']


In [68]:
precision=tp/(tp+fp)
recall=tp/(tp+fn)

f1_score=2*precision*recall/(precision+recall)
print("f1_score::",f1_score)

f1_score:: 0.9673913043478262


In [0]:
print(all_label_ids.numpy()[146:148])

In [0]:
import codecs
file2 = codecs.open("Task1.out","w","utf-8")
pred=list()
for obj in preds:
  #sent=sent.encode('utf-8')
  file2.write(str(obj))
  #pred.append(int(obj.item()))
  file2.write("\n")
file2.close()